# Topological Quantum Error Correction ( TQEC ) : Topological automation

## Setup the basic tools for TQEC

### Simple things to build QEC code with given packages ( can be changed )
- Numpy
- Stim

In [1]:
import numpy as np
import stim
import pymatching

# Create TQEC class 

It takes a tool package ( ex. stim ) and creates an operator set for TQEC platform

In [2]:
# TQEC class
class TQEC:
    
    # Creating quantum circuit 
    # Make memories lists for tracking activated & deactivated qubits
    def __init__( self, pakage_name = stim):
        self.custom_name = pakage_name
        self.circuit = self.custom_name.Circuit()
        
        self.structure_qubits = [ ]
        self.activated_qubits = [ ]
        self.deactivated_qubits = [ ]

        self.measure_num = -1
        self.qubit_measure_dict = { }
    
    # Apply physical qubits into the circuits and give an index for each qubits
    def physical_qubits( self, total_qubit_num ):
        self.structure_qubits = np.arange( total_qubit_num )
        for num in self.structure_qubits:
            self.circuit.append_operation( "QUBIT_COORDS",[ num ] )
    
    # Implementing two qubit gates requires certain qubit ordering [ control1, target1, control2, target2, ... ]
    def merging_lists( self, first_qubit_list, second_qubit_list ):
        return [sub[item] for item in range(len(second_qubit_list))
                        for sub in [first_qubit_list, second_qubit_list]]
    
    ## Whenever we apply quantum gates in the circuit, we update activated & deactivated qubits 
    
    # Implementing cnot gate between given control qubit list and target qubit list
    def cnot( self, control_qubits, target_qubits ):
        self.activated_qubits = list( set(control_qubits).union( target_qubits ) )
        return self.circuit.append_operation( "CNOT", self.merging_lists( control_qubits, target_qubits ) )
    
    # Implementing cz gate between given control qubit list and target qubit list
    def cz( self, control_qubits, target_qubits ):
        self.activated_qubits = list( set(control_qubits).union( target_qubits ) )
        return self.circuit.append_operation( "CZ", self.merging_lists( control_qubits, target_qubits ) )
    
    # Implementing single qubit gate with given gate types ( H, X, Z, ... )
    def single_gate( self, gate_type, activate_qubits ):
        self.activated_qubits = list( set(activate_qubits) )
        return self.circuit.append_operation( gate_type, activate_qubits )

    # Measure qubits 
    def measure( self, activate_qubits, type = 'syn' ):
        
        if type == 'syn':
            # Track all measurement time
            for qubit in activate_qubits[::-1]:
                if qubit in self.qubit_measure_dict.keys():
                    self.qubit_measure_dict[qubit].append(self.measure_num)
                else:
                    self.qubit_measure_dict[qubit] = [ self.measure_num ]
                
                self.measure_num -= 1
                
            self.activated_qubits = list( set(activate_qubits) )
            return self.circuit.append_operation( "MR", activate_qubits )
        
        else:
            current_time = -len(activate_qubits)
            # Track all measurement time
            for qubit in activate_qubits:
                self.qubit_measure_dict[qubit] = [ current_time ]
                current_time += 1
                
            return self.circuit.append_operation( "MR", activate_qubits )
    
    def detector( self, activate_qubits, measure_time = [ 0, 1 ] ):
        
        for qubit in activate_qubits:
            target_rec = [ ]
            for time in measure_time:
                target_rec.append( stim.target_rec( self.qubit_measure_dict[qubit][time] ))
                
            self.circuit.append_operation( "DETECTOR", target_rec )
    
    def detector_final( self, activate_qubits, relative_data_qubits, data_num ):
        
        for qubit in activate_qubits:
            target_rec = [ ]
            target_rec.append( stim.target_rec( self.qubit_measure_dict[qubit][0] - data_num ))
            
            for data in relative_data_qubits:
                target_rec.append( stim.target_rec( self.qubit_measure_dict[data][0] ))
                
            self.circuit.append_operation( "DETECTOR", target_rec )
        
    def observable( self, activate_qubits ):
        
        logic_state = [ ]
        
        # Consider only data qubits
        for qubit in activate_qubits:
            logic_state.append( stim.target_rec( self.qubit_measure_dict[qubit][0] ) )

        self.circuit.append_operation( "OBSERVABLE_INCLUDE", logic_state, 0 )
        
    ### Errors for quantum error correction code
    def bitflip_err( self, activate_qubits, error_rate ):
        return self.circuit.append_operation( "X_ERROR", activate_qubits, error_rate )
    
    def single_dep_err( self, activate_qubits, error_rate ):
        return self.circuit.append_operation( "DEPOLARIZE1", activate_qubits, error_rate )
    
    def two_dep_err( self, control_qubits, target_qubits, error_rate ):
        return self.circuit.append_operation( "DEPOLARIZE2", self.merging_lists( control_qubits, target_qubits ), error_rate )
    
    def idling_err( self, error_rate ):
        return self.circuit.append_operation( "DEPOLARIZE1", list( set(self.structure_qubits) - set( self.activated_qubits ) ), error_rate )

In [3]:
## From Stim package (github:https://github.com/quantumlib/Stim/blob/main/doc/getting_started.ipynb)
def count_logical_errors(circuit: stim.Circuit, num_shots: int) -> int:
    # Sample the circuit.
    sampler = circuit.compile_detector_sampler()
    detection_events, observable_flips = sampler.sample(num_shots, separate_observables=True)

    # Configure a decoder using the circuit.
    detector_error_model = circuit.detector_error_model(decompose_errors=True)
    matcher = pymatching.Matching.from_detector_error_model(detector_error_model)

    # Run the decoder.
    predictions = matcher.decode_batch(detection_events)

    # Count the mistakes.
    num_errors = 0
    for shot in range(num_shots):
        actual_for_shot = observable_flips[shot]
        predicted_for_shot = predictions[shot]
        if not np.array_equal(actual_for_shot, predicted_for_shot):
            num_errors += 1
    return num_errors

In [4]:
if __name__ == "__main__":
    # Create Bell state
    bell_circuit = TQEC()
    circuit = bell_circuit.circuit
    bell_circuit.physical_qubits(2)
    bell_circuit.single_gate("H",[0])
    bell_circuit.cnot([0],[1])
    print(circuit)

QUBIT_COORDS 0
QUBIT_COORDS 1
H 0
CX 0 1


In [5]:
if __name__ == "__main__":
    import stimcirq
    from cirq.contrib.svg import SVGCircuit

    cirq_circuit = stimcirq.stim_circuit_to_cirq_circuit(circuit)
# SVGCircuit(cirq_circuit)